<a href="https://colab.research.google.com/github/gagan3012/notebooks/blob/new/Notebooks/FasttextModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import pandas as pd
import re
import requests as rq
import json
from pandas.io.json import json_normalize


In [ ]:
url = "https://api.yelp.com/v3/businesses/search"

# API key
api = "6NVnd6vwYSReE0eq8yHC4BL-YYjdaZx6UTPzq091R3KbAuAfy0h3BH4Ip5kDBjo5Z7WcYsn4A2tMdZnZJtITRqHPcFVNjQ4xzrL81k_24ElS65l3BXU9ZlovTPyaX3Yx"

#Authorization
headers = { "Authorization": "Bearer " + api }

bus = []

for i in range(0, 1000, 50):
    #passing the parameters to the API.
    params = {"term":"food","location": "Vancouver", "open_now": True, "limit": 50,"offset": i, "c"
          'businesses[x].transactions': ['pickup', 'delivery', 'restaurant_reservation']}
    response = rq.get(url, headers = headers, params = params)
    if response.reason == 'OK':
        bus += response.json()["businesses"]
    else: 
        print(response.reason)

In [ ]:
#Dump the requested JSOn into a variable.
content = json.dumps(bus, indent=4)

#Write the dumped content into a JSON file.
file = open('./yelpBusiness.json', 'w')
file.writelines(content)
file.close()

In [ ]:
data = pd.read_json('yelpBusiness.json')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             310 non-null    object 
 1   alias          310 non-null    object 
 2   name           310 non-null    object 
 3   image_url      310 non-null    object 
 4   is_closed      310 non-null    bool   
 5   url            310 non-null    object 
 6   review_count   310 non-null    int64  
 7   categories     310 non-null    object 
 8   rating         310 non-null    float64
 9   coordinates    310 non-null    object 
 10  transactions   310 non-null    object 
 11  price          254 non-null    object 
 12  location       310 non-null    object 
 13  phone          310 non-null    object 
 14  display_phone  310 non-null    object 
 15  distance       310 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 36.8+ KB


In [ ]:
#To convert the str to json first.
data_cat_1 = data['categories'].to_json(orient='values')

#To import the values as json.
data_cat_2 = pd.read_json(data_cat_1)

#To convert json into json as records.
data_cat_3 = data_cat_2[0].to_json(orient='records')

#Reading from the json.
data_cat = pd.read_json(data_cat_3)

#Selecting only the 'title' column and returning a pandas DataFrame.
data_categories = data_cat[['title']]

#To make sure, that the object is a DataFrame type
print(type(data_categories))

#Renaming the column to 'category'
data_categories.columns = ['category']

data = pd.concat([data, data_categories], axis=1)

data_co_ordinates = json_normalize(data['coordinates'], sep='_')

data = pd.concat([data, data_co_ordinates], axis=1)



data_location = json_normalize(data['location'], sep='_')
data = pd.concat([data, data_location], axis=1)

data = data.drop(columns = ['categories', 'coordinates', 'transactions', 'location'])

data.head()

<class 'pandas.core.frame.DataFrame'>


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,category,category,latitude,longitude,category,latitude,longitude,address1,address2,address3,city,zip_code,country,state,display_address
0,8HH7xRjmjwzkhADZWir_Cw,black-rice-izakaya-vancouver,Black Rice Izakaya,https://s3-media2.fl.yelpcdn.com/bphoto/nncfuv...,False,https://www.yelp.com/biz/black-rice-izakaya-va...,237,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 49.278444, 'longitude': -123.114329}",[],$$,"{'address1': '782 Cambie Street', 'address2': ...",+17783790416,+1 778-379-0416,1691.983928,Japanese,Japanese,49.278444,-123.114329,Japanese,49.278444,-123.114329,782 Cambie Street,None,,Vancouver,V6B 2R5,CA,BC,"[782 Cambie Street, Vancouver, BC V6B 2R5, Can..."
1,F5qWJO27UIzMmSObtD9Zeg,tetsu-sushi-bar-vancouver,Tetsu Sushi Bar,https://s3-media1.fl.yelpcdn.com/bphoto/aPXH03...,False,https://www.yelp.com/biz/tetsu-sushi-bar-vanco...,83,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 49.2917758, 'longitude': -123.135...",[],$$,"{'address1': '775 Denman Street', 'address2': ...",+16044285775,+1 604-428-5775,3460.642508,Japanese,Japanese,49.291776,-123.135266,Japanese,49.291776,-123.135266,775 Denman Street,None,,Vancouver,V6G 2L6,CA,BC,"[775 Denman Street, Vancouver, BC V6G 2L6, Can..."
2,R5G-LpFh2mLUCAOxbvP0eQ,duffins-donuts-vancouver,Duffin's Donuts,https://s3-media1.fl.yelpcdn.com/bphoto/rqgqPD...,False,https://www.yelp.com/biz/duffins-donuts-vancou...,267,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali...",4.0,"{'latitude': 49.2328686, 'longitude': -123.077...",[],$,"{'address1': '1391 41st Avenue E', 'address2':...",+16043255544,+1 604-325-5544,4393.743288,Donuts,Donuts,49.232869,-123.077574,Donuts,49.232869,-123.077574,1391 41st Avenue E,,,Vancouver,V5W 1R7,CA,BC,"[1391 41st Avenue E, Vancouver, BC V5W 1R7, Ca..."
3,_3TI4d58nH8xip1V1u---w,the-naam-vancouver-2,The Naam,https://s3-media3.fl.yelpcdn.com/bphoto/HFeIq1...,False,https://www.yelp.com/biz/the-naam-vancouver-2?...,565,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...",3.5,"{'latitude': 49.2683459, 'longitude': -123.167...",[],$$,"{'address1': '2724 W 4th Avenue', 'address2': ...",+16047387151,+1 604-738-7151,3731.461747,Vegetarian,Vegetarian,49.268346,-123.167062,Vegetarian,49.268346,-123.167062,2724 W 4th Avenue,,,Vancouver,V6K 1R1,CA,BC,"[2724 W 4th Avenue, Vancouver, BC V6K 1R1, Can..."
4,0kB2wCl1sFHf3QA7VIqxwA,lucys-eastside-diner-vancouver,Lucy's Eastside Diner,https://s3-media3.fl.yelpcdn.com/bphoto/NQffpU...,False,https://www.yelp.com/biz/lucys-eastside-diner-...,321,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",3.0,"{'latitude': 49.2609143, 'longitude': -123.100...",[],$$,"{'address1': '2708 Main Street', 'address2': '...",+16045681550,+1 604-568-1550,1148.824163,Diners,Diners,49.260914,-123.100815,Diners,49.260914,-123.100815,2708 Main Street,,,Vancouver,V5T,CA,BC,"[2708 Main Street, Vancouver, BC V5T, Canada]"


In [ ]:
for i in range(len(df)):
  business_id = data["id"][i]
  url = f"https://api.yelp.com/v3/businesses/{business_id}/reviews"
  params = {}
  response_json3 = rq.get(url,params=params,headers=headers)

  if response_json3.reason == 'OK':
          review_info += response_json3.json()['reviews']

  content = json.dumps(review_info, indent=4)

#Write the dumped content into a JSON file.
file = open('./yelpreview.json', 'w')
file.writelines(content)
file.close()

df_r = pd.read_json('yelpreview.json')
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1764 entries, 0 to 1763
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1764 non-null   object
 1   url           1764 non-null   object
 2   text          1764 non-null   object
 3   rating        1764 non-null   int64 
 4   time_created  1764 non-null   object
 5   user          1764 non-null   object
dtypes: int64(1), object(5)
memory usage: 82.8+ KB


In [ ]:
import timeit
from gensim.models import FastText
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    text=str(text)
    text = text.lower()
    text = re.sub(REPLACE_BY_SPACE_RE,' ',text)
    text = re.sub(BAD_SYMBOLS_RE,' ',text)
    text = re.sub("[^a-zA-Z ]", " ", text)
    text = ' '.join(filter(lambda x: x not in STOPWORDS,  text.split()))
    return text

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def make_corpus(data):
    corpus=[]
    data["review_clean"] = data["text"].apply(text_prepare)
    for sentence in data["review_clean"]:
        corpus.append(sentence.strip())
    return corpus

In [ ]:
corpus = make_corpus(df_r)

In [ ]:
# Function to clean corpus
def replace_special_char(word, toBeReplaced, void):
    for elem in toBeReplaced :
        if elem in word :
            word = word.replace(elem, void)
    return word

In [ ]:
def get_recommendation(top, data, scores):
    recommendation = pd.DataFrame(columns = ['descriptions','score'])
    count = 0
    for i in top:
        recommendation.at[count, 'descriptions'],recommendation.at[count, 'score'] =  scores[0][count]
        count += 1
    return recommendation

In [ ]:
# Function to generate similarity scores with fastText
def similarity_score_fastText(df, user_text, corpus, ft_model):
    simlarity_df_list = []
    search_key = user_text
    index_value = 0
    for i in df.index:
        headline = " ".join(corpus[i])
        score = ft_model.wv.most_similar(positive=[search_tag])
        simlarity_df_list.append((search_key, headline, index_value, score))
        index_value += 1
    return simlarity_df_list

In [ ]:
# Function to generate recommendation
def generate_recommendation(data, search_tag, corpus, ft_model):
    # calculate similarity score for all entries using fastText
    simlarity_ft_df = similarity_score_fastText(data, search_tag, corpus, ft_model)
    fastText_recommendation_sorted = pd.DataFrame(simlarity_ft_df).sort_values([3], ascending=False)
    fastText_recommendation_sorted = fastText_recommendation_sorted.iloc[0:output_records,:]
    fastText_recommendation_sorted.reset_index(inplace=True)
    index_ft = fastText_recommendation_sorted[2]
    list_scores = fastText_recommendation_sorted[3]
    recommendation = get_recommendation(index_ft, data, list_scores)
    return recommendation

In [ ]:
# fastText function
def get_fastText_rec(data, corpus, ft_model,search_tag):
    # obtain initial search tag
    global output_records
    output_records = 10
    output_records = int(output_records)
    recommendation = generate_recommendation(data, search_tag, corpus, ft_model)
    return recommendation

In [ ]:
def gen_words(stemmed):
    yield stemmed

In [ ]:
# fastText implementation
start_time = timeit.default_timer()
model = FastText(size=100, window=5, min_count=1, workers=4, sg=1)
model.build_vocab(gen_words(corpus))
model.train(gen_words(corpus), total_examples=model.corpus_count, epochs=model.iter)
search_tag = df.description[0]
reco=model.wv.most_similar(positive=[search_tag])
recommendation = get_fastText_rec(df, corpus, model,search_tag)
elapsed = timeit.default_timer() - start_time
print("\nThe result was generated in "+ str(elapsed) +" seconds.")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):



The result was generated in 11.529281183999956 seconds.


In [ ]:
recommendation

,descriptions,score
0,exploding population percent population growth...,0.999076
1,salvador brazil first capital working protect ...,0.998787
2,new york city excited share experience cities ...,0.998771
3,january togo adopted new national forest polic...,0.99877
4,city seattle set bold goal achieving tree cano...,0.998761
5,raleigh located upper neuse watershed southeas...,0.998752
6,urban tree canopy assessments report highlight...,0.998749
7,salvador humid sub humid climate favors plant ...,0.998742
8,urban tree canopy assessments report highlight...,0.998733
9,paulo populous city brazil covers extensive te...,0.998731
